In [64]:
import magellan as mg

time: 966 µs


In [65]:
A = mg.read_csv(mg.get_install_path()+'/datasets/DBLP_ACM/ACM_cleaned.csv')

time: 14 ms


In [66]:
B = mg.read_csv(mg.get_install_path()+'/datasets/DBLP_ACM/DBLP_cleaned.csv')

time: 15.1 ms


In [67]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
time: 1.32 ms


In [68]:
A1 = A.ix[0:100]
B1 = B.ix[0:100]

time: 2.53 ms


In [69]:
a = A1.values

time: 1.08 ms


In [75]:
np.unique(a[:, 4])

array(['ACM SIGMOD Record', 'ACM Transactions on Database Systems (TODS)',
       'International Conference on Management of Data'], dtype=object)

time: 3.51 ms


In [11]:
import Levenshtein 
def lev(s1, s2):
    if s1 is None or s2 is None:
        return np.NaN
    if pd.isnull(s1) or pd.isnull(s2):
        return np.NaN
    s1 = str(s1)
    s2 = str(s2)
    l1 = float(len(s1))
    l2 = float(len(s2))
    return 1.0 - Levenshtein.distance(s1, s2)/(max(l1, l2))

In [8]:
def my_bb(l, r):
    t1 = l['title']
    t2 = r['title']
    res = lev(t1, t2)
    if res < 0.6:
        return False
    else:
        return True

def fn1(A, B):
    x = []
    for i, l in A.iterrows():
        for j, r in B.iterrows():
            if my_bb(l, r) == True:
                x.append([i, j])
    return x

from collections import OrderedDict
def fn2(A, B):
    dA = OrderedDict()
    dB = OrderedDict()
    for i, l in A.iterrows():
        dA[i] = l
    for i, l in B.iterrows():
        dB[i] = l
    x = []
    for i in dA.keys():
        for j in dB.keys():
            a = dA[i]
            b = dB[j]
            if my_bb(a, b) == True:
                x.append([i, j])
    return x

In [25]:
import pandas as pd
def fn3(A, B):
    dA = OrderedDict()
    dB = OrderedDict()
    h = A.columns
    i = 0
    for l in A.itertuples(index=False):
        dA[i] = OrderedDict(zip(h, l))
        i += 1

    h = B.columns
    i = 0
    for l in B.itertuples(index=False):
        dB[i] = OrderedDict(zip(h, l))
        i += 1
    x = []
    for i in dA.keys():
        for j in dB.keys():
            a = dA[i]
            b = dB[j]
            s1 = a['title']
            s2 = b['title']
            if s1 is None or s2 is None:
                return np.NaN
            if pd.isnull(s1) or pd.isnull(s2):
                return np.NaN
            s1 = str(s1)
            s2 = str(s2)
            l1 = float(len(s1))
            l2 = float(len(s2))
            res =  1.0 - Levenshtein.distance(s1, s2)/(max(l1, l2))
            if res >=  0.6:
                x.append([i, j])

In [36]:
import cython

time: 2.41 ms


In [39]:
%load_ext cython

time: 282 ms


time: 4.41 ms


In [58]:
%%cython
import Levenshtein 
import pandas as pd
import numpy as np
from collections import OrderedDict
def lev(char* s1, char* s2):
    if s1 is None or s2 is None:
        return np.NaN
    if pd.isnull(s1) or pd.isnull(s2):
        return np.NaN
#     s1 = str(s1)
#     s2 = str(s2)
    l1 = float(len(s1))
    l2 = float(len(s2))
    return 1.0 - Levenshtein.distance(s1, s2)/(max(l1, l2))

def my_bb1( l,  r):
    t1 = l['title']
    t2 = r['title']
    res = lev(t1, t2)
    if res < 0.6:
        return False
    else:
        return True

time: 373 ms


In [59]:
def fn4(A, B):
    dA = OrderedDict()
    dB = OrderedDict()
    h = A.columns
    i = 0
    for l in A.itertuples(index=False):
        dA[i] = dict(zip(h, l))
        i += 1

    h = B.columns
    i = 0
    for l in B.itertuples(index=False):
        dB[i] = dict(zip(h, l))
        i += 1
    x = []
    for i in dA.keys():
        for j in dB.keys():
            res = my_bb1(dA[i], dB[j])
            if res == True:
                x.append([i, j])

time: 11.3 ms


In [16]:
%lprun -f test_profile.fn1 test_profile.fn1(A1, B1)

In [26]:
%lprun -f fn3 fn3(A1, B1)

In [49]:
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 1.37 ms


In [60]:
%lprun -f fn4 fn4(A1, B1)

time: 15.4 s


In [61]:
%prun fn4(A1, B1)

 time: 11.7 s


In [30]:
fn3(A1, B1)

time: 11.1 s


In [32]:
x = fn2(A1, B1)

time: 21.1 s
